# Predictive Models

In the last two notebooks we had a look at two of the components of the Basis Mixer. In this notebook we add the third part of the puzzle: the **Predictive Models**.

A predictive model is defined as a mathematical which maps score information (encoded by the basis functions) $\mathbf{\Phi}$ to expressive parameters $\mathbf{Y}$

$$F(\boldsymbol{\Phi}) = \mathbf{Y}$$

## Defining and Building Predictive Models

In [1]:
%matplotlib notebook
import numpy as np
import os
from basismixer import make_datasets
from helper import init_dataset, data
from helper.predictions import construct_model, setup_output_directory, train_model, split_dataset

In [2]:
out_dir = setup_output_directory()

The configuration of the full predictive model can be defined as a list of models defined for individual (or groups) of expressive parameters.

The structure of this configuration can be summarized as follows:

* `input_type`: specifies if the model predicts *notewise* or *onsetwise* parameters.
* `basis_functions`: A list with the basis functions (or the familiy of basis functions) used to specify the model. Alternatively, we can use a dataset to specify the basis functions.
* `parameter_names`: Name of the expressive parameters
* `model`: A dictionary specifying the architecture of the model
    * `constructor`: A list specifying the module and method used to construct the model
    * `args`: Arguments for the constructor.
* `train_args`: A dictionary containing parameters for training the model.

Let us define an example of such an architecture

In [3]:
model_config = [
    dict(onsetwise=False,
         basis_functions=['polynomial_pitch_basis',
                          'loudness_direction_basis',
                          'tempo_direction_basis',
                          'articulation_basis',
                          'duration_basis',
                          # my_basis,
                          'slur_basis',
                          'fermata_basis',
                          'metrical_basis'],
         parameter_names=['velocity_dev', 'timing', 'articulation_log'],
         seq_len=1,
         model=dict(constructor=['basismixer.predictive_models', 'FeedForwardModel'],
                    args=dict(hidden_size=128)),
         train_args=dict(
             optimizer=['Adam', dict(lr=1e-4)],
             epochs=1,
             save_freq=1,
             early_stopping=100,
             batch_size=1000,
         )
    ),
    dict(onsetwise=True,
         basis_functions=['polynomial_pitch_basis',
                          'loudness_direction_basis',
                          'tempo_direction_basis',
                          'articulation_basis',
                          'duration_basis',
                          'slur_basis',
                          'fermata_basis',
                          'metrical_basis'],
         parameter_names=['velocity_trend', 'beat_period_standardized', 'beat_period_mean', 'beat_period_std'],
         seq_len=200,
         model=dict(constructor=['basismixer.predictive_models', 'RecurrentModel'],
                    args=dict(recurrent_size=128,
                              n_layers=1,
                              hidden_size=64)),
         train_args=dict(
             optimizer=['Adam', dict(lr=1e-4)],
             epochs=1,
             save_freq=1,
             early_stopping=100,
             batch_size=20,
         )
    )
]

## 2. Making the dataset

In [4]:
init_dataset() # download the corpus if necessary; set some variables

# path to the MusicXML and Match files
xmlfolder = os.path.join(data.DATASET_DIR, 'musicxml')
matchfolder = os.path.join(data.DATASET_DIR, 'match')

Output()

In [5]:
datasets = make_datasets(model_config,
                         xmlfolder,
                         matchfolder)

[WARNING] Dropping slur 1 starting at 80 (n33-1) and ending at 80 (n36-1)
[WARNING] Dropping slur 1 starting at 104 (n37a-1) and ending at 80 (n36-1)
[WARNING] Dropping slur 1 starting at 272 (n33-2) and ending at 272 (n36-2)
[WARNING] Dropping slur 1 starting at 296 (n37a-2) and ending at 272 (n36-2)
[WARNING] Dropping slur 1 starting at 432 (n96-1) and ending at 432 (n98-1)
[WARNING] Dropping slur 1 starting at 440 (n106-1) and ending at 432 (n98-1)
[WARNING] Dropping slur 1 starting at 624 (n96-2) and ending at 624 (n98-2)
[WARNING] Dropping slur 1 starting at 632 (n106-2) and ending at 624 (n98-2)
[WARNING] Dropping slur 3 starting at 1632 (n120-1) and ending at 1632 (n122-1)
[WARNING] Dropping slur 3 starting at 2592 (n120-2) and ending at 1696 (n126-1)
[WARNING] Dropping slur 3 starting at 2592 (n120-2) and ending at 2592 (n122-2)



## Training the models

Given a training set of expressive performances aligned to their scores, we can train the models in a supervised way by minimizing the *mean squared error* between predictions and the observed expressive parameters.

In [6]:
models = []
test_sets = []
for (dataset, in_names, out_names), config in zip(datasets, model_config):
    
    # Build model
    model, model_out_dir = construct_model(config, in_names, out_names, out_dir)
    # Split datasets
    train_set, valid_set, test_set = split_dataset(dataset)
    # Train Model
    train_model(model, train_set, valid_set, config, model_out_dir)
    
    models.append(model)
    test_sets.append(test_set)


[WARNING] No GPU available! Training will be performed on CPU.
epoch: 0/1:  11%|█         | 3/28 [00:00<00:01, 21.77it/s]

Pieces per dataset
Train set:	57
Test set:	17
Validation set:	14

[79 62  5 47  4  0  7 58 50 70 26 24 31 33 63 85 81 23 40 29 67 55 76 78
 12 75 48 86 20 37 43 59 34 52  6 60 71 80 84 44 49 11 54 39 69 14  1 83
  9  2 21 46 10  8 38 72 25]
[51 82  3 65 27 57 35 19 15 17 73 18 56 66]
[74 77 16 53 28 87 42 32 45 41 68 30 61 22 13 36 64]
27881 6808 8673


epoch: 0/1: 100%|██████████| 28/28 [00:01<00:00, 19.86it/s]
[WARNING] No GPU available! Training will be performed on CPU.
epoch: 0/1:   0%|          | 0/1 [00:00<?, ?it/s]

Pieces per dataset
Train set:	57
Test set:	17
Validation set:	14

[21 42 23 15 39 44 49 82 81 60 62 16  6 58 61 32 74 84 54 70 41 86 50 53
 64 46 13  9 48 67 27 34 77 45 57 17 19 29  4 36 25 28  7 47  1 87 55  3
 71 56 51 30 43  8 68 65 66]
[18 22 72 26 20 85 35 40 73 10 79 33 83 76]
[63 80 78 69 31  2 38 11 37 24  0 14  5 12 75 52 59]
18 0 3


epoch: 0/1: 100%|██████████| 1/1 [00:00<00:00,  2.76it/s]
/Users/aae/miniconda3/envs/basismixer/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/aae/miniconda3/envs/basismixer/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: invalid index to scalar variable.

In [ ]:
# plot_predictions(models, targets)
print(len(train_set))